<a href="https://colab.research.google.com/github/HedersonSantos/Noticias/blob/main/preProcessamento/preProcessamentoFSP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
import pandas as pd
import numpy as np
from google.colab import files, drive
from pathlib import Path
from sklearn.model_selection import train_test_split
import io, re, os
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('max_colwidth', None)

### Montando Google Driver e abrindo arquivos de noticias

o arquivo articles.csv contém 167053 noticias da Folha de São Paulo. Este dataset está disponível em .... <br>
O arquivo news.csv contém xxx noticias extraídos dos sites UOL, G1, GoogleNews, Poder360 através de web scrapping cujo algoritmo foi montado por nós.

In [21]:
drive.mount('/gdrive')
%cd /gdrive/My\ Drive/Colab\ Notebooks/nlp_tcc
path = '/gdrive/My Drive/Colab Notebooks'
KEY='4/1AX4XfWjO8LFULyUVA2hr76Ekj281Rn58ieVYw_BkgE4HYvy9Lf8PJ1W3aOQ'

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive/My Drive/Colab Notebooks/nlp_tcc


In [ ]:

#!rm -R amostra_news_integrada


In [ ]:
#!unzip noticiasFSP.zip
!ls 

amostra_news_integrada	bertimbau_test	     noticia_tratada.csv
articles.csv		news.csv
bertimbau_base_results	news_integradas.csv


In [ ]:
!ls  -lh

total 522M
drwx------ 2 root root 4.0K Jul 31 21:17  amostra_news_integrada
-rw------- 1 root root  57K Jul 13 19:50  artigosWikibr.ipynb
-rw------- 1 root root  37K Aug  1 23:19  BERTimbau_com_amostras.ipynb
drwx------ 2 root root 4.0K Aug  7 13:00  bertimbau_resp
-rw------- 1 root root  98K Aug  7 15:29 'Cópia de BERTimbau_Testes (1).ipynb'
-rw------- 1 root root  72K Aug  1 12:19 'Cópia de BERTimbau_Testes (2).ipynb'
-rw------- 1 root root  46K Aug  1 11:45 'Cópia de BERTimbau_Testes (3).ipynb'
-rw------- 1 root root 170K Aug  7 17:47 'Cópia de BERTimbau_Testes.ipynb'
-rw------- 1 root root 521M Aug  1 14:59  news_integradas.csv
-rw------- 1 root root  306 Jul 12 18:08  Untitled
-rw------- 1 root root 1.2K Jul 12 17:26  Untitled1.ipynb


**Conjunto de Notícias da Folha de São Paulo**

In [ ]:
dfFSP = pd.read_csv('articles.csv', header=0)
print(dfFSP.columns)
dfFSP.shape

Index(['title', 'text', 'date', 'category', 'subcategory', 'link'], dtype='object')


(167053, 6)

In [ ]:
dfFSP.head(1)

,title,text,date,category,subcategory,link
0,"Lula diz que está 'lascado', mas que ainda tem força como cabo eleitoral","Com a possibilidade de uma condenação impedir sua candidatura em 2018, o ex-presidente Luiz Inácio Lula da Silva fez, nesta segunda (9), um discurso inflamado contra a Lava Jato, no qual disse saber que está ""lascado"", exigiu um pedido de desculpas do juiz Sergio Moro e afirmou que, mesmo fora da disputa pelo Planalto, será um cabo eleitoral expressivo para a sucessão de Michel Temer. Segundo o petista, réu em sete ações penais, o objetivo de Moro é impedir sua candidatura no ano que vem, desidratando-o, inclusive, no apoio a um nome alternativo, como o do ex-prefeito de São Paulo Fernando Haddad (PT), caso ele não possa concorrer à Presidência. ""Eu sei que tô lascado, todo dia tem um processo. Eu não quero nem que Moro me absolva, eu só quero que ele peça desculpas"", disse Lula durante um seminário sobre educação em Brasília. ""Eles [investigadores] chegam a dizer: 'Ah, se o Lula não for candidato, ele não vai ter força como cabo eleitoral'. Testem"", completou o petista. Para o ex-presidente, Moro usou ""mentiras contadas pela Polícia Federal e pelo Ministério Público"" para julgá-lo e condená-lo a nove anos e seis meses de prisão pelo caso do tríplex em Guarujá (SP). O ex-presidente disse ainda não ter ""medo"" dos investigadores que, de acordo com ele, estão acostumados a ""mexer com deputados e senadores"" que temem as apurações. ""Eu quero que eles saibam o seguinte: se eles estão acostumados a lidar com deputado que tem medo deles, a mexer com senadores que têm medo deles, quero dizer que tenho respeito profundo por quem me respeita, pelas leis que nós ajudamos a criar, mas não tenho respeito por quem não me respeita e eles não me respeitaram"", afirmou o petista. De acordo com aliados, Lula não gosta de discutir, mesmo que nos bastidores, a chance de não ser candidato ao Planalto e a projeção do nome de Haddad como plano B do PT tem incomodado os mais próximos ao ex-presidente. O ex-prefeito, que estava no evento nesta segunda, fez um discurso rápido, de menos de dez minutos, em que encerrou dizendo esperar que Lula assuma a Presidência em 2019. ""Espero que dia 1º de janeiro de 2019 esse pesadelo chamado Temer acabe e o senhor assuma a Presidência da República"", disse Haddad. 'DEMÔNIO DO MERCADO' Lula voltou a fazer um discurso mais agressivo em relação ao mercado e disse que ""não tem cara de demônio"", mas quer que o respeitem ""como se fosse"". ""Não tenho cara de demônio, mas quero que eles me respeitem como se eu fosse, porque eles sabem que a economia não vai ficar subordinada ao elitismo da sociedade brasileira"", disse o ex-presidente. O petista rivalizou ainda com o deputado Jair Bolsonaro (PSC-RJ), segundo colocado nas últimas pesquisas empatado com Marina Silva, e disse que se ele ""agrada ao mercado"", o PT tem que ""desagradar"". A Folha publicou nesta segunda (9) reportagem em que mostrou que o deputado ensaia movimento ao centro no debate econômico, adotando um discurso simpático aos investidores do mercado financeiro.",2017-09-10,poder,NaN,http://www1.folha.uol.com.br/poder/2017/10/1925743-lula-diz-que-esta-lascado-mas-que-ainda-tem-forca-como-cabo-eleitoral.shtml


**Quantidade de noticias por categoria**

In [ ]:
#['title', 'text', 'date', 'category', 'subcategory', 'link'
categoriaFSP = dfFSP.groupby(['category'])['link'].count().reset_index().sort_values(by = 'link', ascending=False)
categoriaFSP

,category,link
35,poder,22022
9,colunas,21622
26,mercado,20970
18,esporte,19730
29,mundo,17130
12,cotidiano,16967
23,ilustrada,16345
33,opiniao,4525
34,paineldoleitor,4011
37,saopaulo,3955


**Conjunto de Notícias da nossa base de dados**

In [ ]:
dfNews = pd.read_csv('news.csv', header=0)
print(dfNews.columns)
dfNews.shape

Index(['FONTE', 'CATEGORIA', 'TEXTO', 'URL', 'PUBLISHEDAT', 'TITLE', 'AUTHOR',
       'DT_REGISTRO', 'TEXT_TRATADO'],
      dtype='object')


(11176, 9)

In [ ]:
'''tornando o nome das colunas dos dois datasets iguais '''
dfNews = dfNews.rename(columns={'CATEGORIA':'category',
                       'URL':'link',
                       'PUBLISHEDAT':'date',
                       'TITLE':'title',
                       'TEXT_TRATADO':'text',
                       'FONTE':'fonte'}).copy()
dfNews = dfNews[['title', 'text', 'date', 'category', 'link','fonte']]


    

In [ ]:
dfNews.head(1)

,title,text,date,category,link,fonte
0,"Putin não espera ""grandes avanços"" em cúpula com Biden","Em entrevista para as principais agências de notícias internacionais, incluindo a Efe, Putin enfatizou que os dois países, apesar das discordâncias, têm interesses em comum e que ele está confiante que a cúpula com Biden será construtiva.\nEntre as áreas de interesse comum, o chefe do Kremlin citou a solução de problemas ambientais, segurança estratégica e a dissolução de uma série de crises regionais.\n""Isso é muito importante, porque tem a ver com a segurança de nossos países"", afirmou Putin, além de destacar, por exemplo, que a crise no Oriente Médio ""leva ao crescimento do terrorismo"".\n""(Isso é) muito perigoso para todos, para os Estados Unidos, para a Rússia, para a Europa, para outras regiões do mundo"", acrescentou.\nPutin enfatizou a importância da segurança estratégica e garantiu que a Rússia não quer ""assustar ninguém"" com seus novos armamentos.\n""Sim, estamos desenvolvendo esses armamentos e conseguimos certos resultados"", disse o mandatário, para quem a Rússia está um passo ""à frente"" de outros países.\nEle também admitiu que os Estados Unidos e outras potências logo alcançarão resultados semelhantes, e portanto ""é melhor pensar em como vamos viver juntos"".\n\nO Conselho Tutelar de\nIvaiporã foi acionado por volta das 12h30 de sexta-feira (4) para acompanhar\nalgumas...\nO sábado (5) começa com tempo estável nas regiões paranaenses. Ao amanhecer, Apucarana e Arapongas registraram...\nO Senado vai analisar a Medida Provisória 1.034/2021, que aumenta a tributação de instituições financeiras O texto \nEm meio à pandemia da Covid-19, especialistas apontam que a menor frequência nos parques públicos pode estar...\nA Empresa de Limpeza Urbana de Salvador (Limpurb) fará uma série de ações até 11 de junho como parte da campanha...\nOs ministros das Finanças do G7 esperam anunciar um acordo neste sábado (5) sobre um imposto mínimo global e uma...",2021-06-05T03:00:00Z,mundo,https://noticias.uol.com.br/ultimas-noticias/efe/2021/06/05/putin-nao-espera-grandes-avancos-em-cupula-com-biden.htm,UOL


In [ ]:
categoriaNossa = dfNews.groupby(['category'])['link'].count().reset_index().sort_values(by = 'link', ascending=False)

### Integrando as duas bases de dados de noticias

In [ ]:
dfFSP['fonte']='FolhaSP'
dfnoticias = pd.concat([dfFSP,dfNews],sort=False)
dfnoticias.shape

(178229, 7)

In [ ]:
dfnoticias['fonte'].value_counts()

FolhaSP           167053
UOL                 4034
google-news-br      2089
Poder360            1448
Notícias da TV      1262
Name: fonte, dtype: int64

In [ ]:
deParaCategorias = pd.merge(categoriaFSP[['category','link']], categoriaNossa, on=['category'], how='outer')

**Categorias com mesmo nome nos dois conjunto de dados**

In [ ]:
deParaCategorias[(~deParaCategorias['link_x'].isnull()) & (~deParaCategorias['link_y'].isnull()) ]

,category,link_x,link_y
1,colunas,21622.0,263.0
3,esporte,19730.0,590.0
4,mundo,17130.0,1251.0
5,cotidiano,16967.0,77.0
12,educacao,2118.0,27.0


**Categorias que estão no nosso dataset e não estão no dataset da FSP**

In [ ]:
str(deParaCategorias[(deParaCategorias['link_x'].isnull()) ]['category'].tolist())

"['midia', 'local', 'politica', 'novelas', 'miscelanea', 'saude', 'economia', 'brasil', 'coronavirus', 'justica', 'celebridades', 'tecnologia', 'fantastico', 'meio-ambiente', 'podcast', 'eleicoes', 'loterias', 'casos-de-policia', 'olimpiadas', 'pesquisas', 'confere', 'enem', 'carros', 'agronegocios', 'pme', 'mais', 'tilt, reuters', 'portuguese', 'legislacao', 'atitude', 'internet', 'coluna', 'especiais', 'start', 'sociedade, educacao', 'realities', 'programas, 3-em-1', 'comprova', 'pix', 'combate', 'transito', 'nieman', 'motor', 'inovacao', 'historia', 'nossa', 'carros, colunas', 'boxe', 'tilt, estado', 'brics', 'mais, levante', 'sociedade, epoca', 'saude, estado', 'eurocopa', 'rio, rio', 'golfe', 'programas', 'carros, colunas, infomoto', 'justica, 3-dos-ex-pms-levados-por-salles-para-ministerio-foram-afastados-pelo-stf', 'exercito', 'diplomacia', 'redacao']"

In [ ]:
str(categoriaFSP['category'].tolist())

"['poder', 'colunas', 'mercado', 'esporte', 'mundo', 'cotidiano', 'ilustrada', 'opiniao', 'paineldoleitor', 'saopaulo', 'tec', 'tv', 'educacao', 'turismo', 'ilustrissima', 'ciencia', 'equilibrioesaude', 'sobretudo', 'bbc', 'folhinha', 'empreendedorsocial', 'comida', 'asmais', 'ambiente', 'seminariosfolha', 'serafina', 'o-melhor-de-sao-paulo', 'vice', 'guia-de-livros-discos-filmes', 'topofmind', 'banco-de-dados', 'dw', 'especial', 'infograficos', 'cenarios-2017', 'rfi', 'guia-de-livros-filmes-discos', 'multimidia', 'treinamento', 'treinamentocienciaesaude', 'mulher', 'euronews', 'ombudsman', 'contas-de-casa', '2016', 'bichos', 'musica', '2015']"

DE/PARA de categorias.

nossa             FSP<br>
midia      <--    tv, multimidia, novelas, celebridades   <br>
local      <--    'saopaulo', 'o-melhor-de-sao-paulo' <br>
politica      <--    poder <br>
miscelanea -->     <br>
saude      <--   equilibrioesaude <br>
economia   <--   mercado <br>
brasil                   <br>
olimpiadas -->  esporte <br>
tecnologia  <-- tec,'banco-de-dados', 'dw'  <br>
meio-ambiente <-- ambiente <br>
enem --> educacao








In [ ]:
dfnoticias['category'] = dfnoticias.loc[:,['category']].apply(lambda x: 'midia' if x['category'] in ['tv', 'multimidia', 'novelas', 'celebridades'] else 
                                                                        'local' if x['category'] in [ 'saopaulo', 'o-melhor-de-sao-paulo'] else 
                                                                        'politica' if x['category'] in ['poder'] else
                                                                        'saude' if x['category'] in ['equilibrioesaude'] else 
                                                                        'economia' if x['category'] in ['mercado'] else
                                                                        'esporte' if x['category'] in ['olimpiadas'] else
                                                                        'tecnologia' if x['category'] in [ 'tec','banco-de-dados', 'dw'] else
                                                                        'meio-ambiente' if x['category'] in [ 'ambiente'] else 
                                                                        'educacao' if x['category'] in ['enem'] else
                                                                        x['category'], axis=1)


**Quantitativo de noticias por categoria**

In [ ]:
dfnoticias['category'].value_counts()

politica                                                                             23499
colunas                                                                              21885
economia                                                                             21476
esporte                                                                              20331
mundo                                                                                18381
cotidiano                                                                            17044
ilustrada                                                                            16345
local                                                                                 5628
midia                                                                                 4817
opiniao                                                                               4525
paineldoleitor                                                                        4011

Limpeza em Textos extraídos pela nossa rotina de Web Scrapping
Foram retirados primeiros paragráfos duplicados - ocorrência referente à fonte google-news-br -, textos de chamadas a outras noticias e chamadas a vídeos de outras reportagens no Youtube - ocorrências referentes à fonte UOL.   

In [ ]:
class preprocessamento():
  def limpaReferenciaOutrasNoticiasApos2enters(self,dfnews,fonte):
    '''extrai links de outras noticias no texto da noticial principal do uol. Considera quando inicia com dois enters'''
    for index, row in dfnews[dfnews['fonte']==fonte][['fonte','category','text','link']].iterrows():
      if '...' in str(row['text']):
        for match in re.finditer(r'(?s)((?:[^\n][\n]?)+)', row['text']):
          if not ('...' in str(row['text'])[match.start(): match.end()]):
            dfnews.loc[dfnews['link']==row['link'],['text']]=row['text'][match.start(): match.end()]
    return dfnews
  def limpaReferenciaOutrasNoticiasApos1enter(self,dfnews,fonte):
    '''extraindo links de outras noticias no texto da noticial principal do uol. obtem paragrafo por paragrafo e verifica se nele tem ...'''
    for index, row in dfnews[dfnews['fonte']==fonte].iterrows():
      if '...' in str(row['text']):
        texto=''
        for match in re.finditer(r'((?:[^\n][\.]?)+)', row['text']):
          if not ('...' in str(row['text'])[match.start(): match.end()]):
            texto+=str(row['text'])[match.start(): match.end()] + '\n'
          else:  
            break
        dfnews.loc[dfnews['link']==row['link'],['text']]=texto
    return dfnews

  def limpaReferenciaVideosYoutubeSemContexto(self,dfnews,fonte):
    '''retira do final do texto referências a Youtube '''
    for index, row in dfnews[(dfnews['fonte']=='UOL')&(dfnews['text'].str.contains('Já assistiu aos nossos novos vídeos no YouTube'))].iterrows():
      for match in re.finditer(r'(Leia outras reportagens da série aqui)|(Já assistiu aos nossos novos vídeos no YouTube)', row['text']):
        dfnews.loc[dfnews['link']==row['link'],['text']]=str(row['text'])[0: match.start()]
        break
    return dfnews
  def limpaParagrafosDuplicados(self,dfnews,fonte):
    '''excluindo primeiro paragráfo duplicado '''
    for index, row in dfnews[dfnews['fonte']==fonte].iterrows():
      texto=''
      texto_anterior=''
      for match in re.finditer(r'((?:[^\n][\.]?)+)', row['text']):
        if (row['text'][match.start():match.end()])!=(texto_anterior):
          texto+=row['text'][match.start():match.end()]+'\n'
          texto_anterior=row['text'][match.start():match.end()]
      dfnews.loc[dfnews['link']==row['link'],['text']]=texto
      return dfnews
  def numerandoCategorias(self,dfnews):
    dfNewsGrupo = dfnews.groupby(['category'])['link'].count().reset_index().sort_values('link',ascending=False)
    dictCategoriaNumero = dict(zip(dfNewsGrupo['category'],range(0,dfNewsGrupo.shape[0])))
    dfnews['category_nro'] = dfnews['category'].map(dictCategoriaNumero)
    return dfnews








In [ ]:
pp=preprocessamento()
dfnews_tratadas = dfnoticias.copy()
dfnews_tratadas=pp.limpaReferenciaOutrasNoticiasApos2enters(dfnews_tratadas,'UOL')
dfnews_tratadas=pp.limpaReferenciaOutrasNoticiasApos1enter(dfnews_tratadas,'UOL')  
dfnews_tratadas=pp.limpaReferenciaVideosYoutubeSemContexto(dfnews_tratadas,'UOL')
dfnews_tratadas=pp.limpaParagrafosDuplicados(dfnews_tratadas,'google-news-br')
dfnews_tratadas=pp.numerandoCategorias(dfnews_tratadas)

In [ ]:
dfnews_tratadas.to_csv("news_integradas.csv",header=True,doublequote=True, index=False, quotechar='"', encoding='utf-8')
#Este arquivo está no drive google hederson.santos@gmail.com

NameError: ignored

In [ ]:
#!!unzip news_integradas.zip

Archive:  news_integradas.zip
  inflating: news_integradas.csv     


In [22]:
dfnews_tratadas = pd.read_csv("./news_integradas.csv",header=0,doublequote=True, quotechar='"', encoding='utf-8')
print(dfnews_tratadas.shape)

(178229, 8)


In [ ]:
dfnews_tratadas.head(1)

,title,text,date,category,subcategory,link,fonte,category_nro
0,"Lula diz que está 'lascado', mas que ainda tem força como cabo eleitoral","Com a possibilidade de uma condenação impedir sua candidatura em 2018, o ex-presidente Luiz Inácio Lula da Silva fez, nesta segunda (9), um discurso inflamado contra a Lava Jato, no qual disse saber que está ""lascado"", exigiu um pedido de desculpas do juiz Sergio Moro e afirmou que, mesmo fora da disputa pelo Planalto, será um cabo eleitoral expressivo para a sucessão de Michel Temer. Segundo o petista, réu em sete ações penais, o objetivo de Moro é impedir sua candidatura no ano que vem, desidratando-o, inclusive, no apoio a um nome alternativo, como o do ex-prefeito de São Paulo Fernando Haddad (PT), caso ele não possa concorrer à Presidência. ""Eu sei que tô lascado, todo dia tem um processo. Eu não quero nem que Moro me absolva, eu só quero que ele peça desculpas"", disse Lula durante um seminário sobre educação em Brasília. ""Eles [investigadores] chegam a dizer: 'Ah, se o Lula não for candidato, ele não vai ter força como cabo eleitoral'. Testem"", completou o petista. Para o ex-presidente, Moro usou ""mentiras contadas pela Polícia Federal e pelo Ministério Público"" para julgá-lo e condená-lo a nove anos e seis meses de prisão pelo caso do tríplex em Guarujá (SP). O ex-presidente disse ainda não ter ""medo"" dos investigadores que, de acordo com ele, estão acostumados a ""mexer com deputados e senadores"" que temem as apurações. ""Eu quero que eles saibam o seguinte: se eles estão acostumados a lidar com deputado que tem medo deles, a mexer com senadores que têm medo deles, quero dizer que tenho respeito profundo por quem me respeita, pelas leis que nós ajudamos a criar, mas não tenho respeito por quem não me respeita e eles não me respeitaram"", afirmou o petista. De acordo com aliados, Lula não gosta de discutir, mesmo que nos bastidores, a chance de não ser candidato ao Planalto e a projeção do nome de Haddad como plano B do PT tem incomodado os mais próximos ao ex-presidente. O ex-prefeito, que estava no evento nesta segunda, fez um discurso rápido, de menos de dez minutos, em que encerrou dizendo esperar que Lula assuma a Presidência em 2019. ""Espero que dia 1º de janeiro de 2019 esse pesadelo chamado Temer acabe e o senhor assuma a Presidência da República"", disse Haddad. 'DEMÔNIO DO MERCADO' Lula voltou a fazer um discurso mais agressivo em relação ao mercado e disse que ""não tem cara de demônio"", mas quer que o respeitem ""como se fosse"". ""Não tenho cara de demônio, mas quero que eles me respeitem como se eu fosse, porque eles sabem que a economia não vai ficar subordinada ao elitismo da sociedade brasileira"", disse o ex-presidente. O petista rivalizou ainda com o deputado Jair Bolsonaro (PSC-RJ), segundo colocado nas últimas pesquisas empatado com Marina Silva, e disse que se ele ""agrada ao mercado"", o PT tem que ""desagradar"". A Folha publicou nesta segunda (9) reportagem em que mostrou que o deputado ensaia movimento ao centro no debate econômico, adotando um discurso simpático aos investidores do mercado financeiro.",2017-09-10,politica,NaN,http://www1.folha.uol.com.br/poder/2017/10/1925743-lula-diz-que-esta-lascado-mas-que-ainda-tem-forca-como-cabo-eleitoral.shtml,FolhaSP,0


In [23]:
dfNewsGrupo = dfnews_tratadas.groupby(['category','category_nro'])['link'].count().reset_index().sort_values(by='link',ascending=False).reset_index().rename(columns={'link':'qtd_registros'})
dfNewsGrupo

,index,category,category_nro,qtd_registros
0,70,politica,0,23499
1,17,colunas,1,21885
2,26,economia,2,21476
3,32,esporte,3,20331
4,59,mundo,4,18381
5,24,cotidiano,5,17044
6,42,ilustrada,6,16345
7,50,local,7,5628
8,55,midia,8,4817
9,64,opiniao,9,4525


In [24]:
def gravaArquivos(dfAmostra, caminho):
  df_train, df_test_valid = train_test_split(dfAmostra, train_size = 0.8, random_state = 10)
  df_valid, df_test = train_test_split(df_test_valid, train_size = 0.5, random_state = 10)
  print('Treinamento:',df_train.shape)
  print('Teste:', df_test.shape)
  print('Valid:', df_valid.shape)
  df_train.to_csv(caminho + '/train.csv',header=True,doublequote=True, index=False, quotechar='"', encoding='utf-8')
  df_test.to_csv(caminho + '/test.csv',header=True,doublequote=True, index=False, quotechar='"', encoding='utf-8')
  df_valid.to_csv(caminho + '/valid.csv',header=True,doublequote=True, index=False, quotechar='"', encoding='utf-8')
def gravaArquivoUnico(dfAmostra, caminho):
  print('Dimensao:',dfAmostra.shape)
  os.makedirs(caminho,exist_ok=True )
  dfAmostra.to_csv(caminho + '/amostra.csv',header=True,doublequote=True, index=False, quotechar='"', encoding='utf-8')
  

In [ ]:
n_registros=[[1,1000]]  #considerar categorias com quantidade de registros acima desta. Se zero considerar toda a amostra.
tam_amostra =[0]  #tamanho da amostra a ser selecionada em cada categoria. Se zero, pega toda a quantidade da categoria.
quantidade = False #determina se obtem uma fração ou uma quantidade de cada categoria. Se False obtem a fraçao.
idx=5 #nro do dataset
arquivoUnico = True # Se False faz o split em Treinamento, Validacao e Teste
colunas = ['text','category','category_nro']
for registro, amostra in zip(n_registros, tam_amostra):
  categorias = dfNewsGrupo[(dfNewsGrupo['qtd_registros']>=registro[0]) & (dfNewsGrupo['qtd_registros']<registro[1]) ].loc[:,['category']]['category'].to_list()
  print('qtd_categorias:',len(categorias))
  frame=[]
  if registro[0]>0:
    for categoria in categorias:
      if amostra==0:
        dftmp = dfNewsGrupo[dfNewsGrupo['category']==categoria]
        amostra_l = dftmp['qtd_registros'].tolist()[0]-1
        quantidade=True
      else:
        amostra_l = amostra
      if quantidade==True:
        dftmp = dfnews_tratadas[dfnews_tratadas['category']==categoria].loc[:,colunas].sample(n=amostra_l, random_state=42)
      else:
        dftmp = dfnews_tratadas[dfnews_tratadas['category']==categoria].loc[:,colunas].sample(frac=amostra_l, random_state=42)
      frame.append(dftmp)
    dfAmostra = pd.concat(frame,ignore_index=True, sort=False)
  else:
    if quantidade==True:
      dfAmostra = dfnews_tratadas.loc[:,colunas].sample(n=amostra, random_state=42) #usar pesos????
    else:
      dfAmostra = dfnews_tratadas.loc[:,colunas].sample(frac=amostra, random_state=42) #usar pesos????
  if not dfAmostra.empty:
    if not arquivoUnico == True:
      gravaArquivos(dfAmostra,'./amostra_news_integrada/amostra_'+str(idx))
    else:
      gravaArquivoUnico(dfAmostra,'./amostra_news_integrada/amostra_'+str(idx))
    print('Amostra', idx,':', dfAmostra.shape)
  else:
    print('Amostra ',idx,' não gerada.')
  idx+=1


qtd_categorias: 77
Dimensao: (7986, 2)
Amostra 6 : (7986, 2)


444

In [ ]:
!ls ./amostra_news_integrada/amostra_4/

amostra.csv  dfPreProc_25.csv  dfPreProc.csv


In [ ]:
df4 = pd.read_csv('./amostra_news_integrada/amostra_4/amostra.csv')
df5 = pd.read_csv('./amostra_news_integrada/amostra_5/amostra.csv')
df6 = pd.read_csv('./amostra_news_integrada/amostra_6/amostra.csv')
print(df4.shape, df5.shape, df6.shape)
dfT = pd.concat([df4, df5, df6], sort=False)
gravaArquivoUnico(dfT,'./amostra_news_integrada/amostra_4')

(7986, 2) (6241, 2) (7986, 2)
Dimensao: (22213, 2)


In [ ]:
dfT = pd.read_csv('./amostra_news_integrada/amostra_4/amostra.csv')
print(dfT.shape)
dfT['category_nro'].value_counts()

(22213, 2)


18    1958
19    1750
20    1680
21    1654
22    1534
7     1126
23    1124
24    1094
8      963
9      905
25     886
10     802
26     756
27     694
28     666
11     492
12     430
29     408
13     382
14     381
30     290
31     284
15     282
16     267
17     211
32     170
33     156
34     100
35      92
37      84
36      84
38      84
39      56
40      56
41      54
42      40
43      34
44      30
45      26
46      14
47      14
48      12
49       8
50       8
53       6
52       6
51       6
58       4
54       4
57       4
56       4
59       4
55       4
64       2
72       2
74       2
65       2
62       2
67       2
66       2
71       2
61       2
68       2
73       2
69       2
60       2
70       2
63       2
Name: category_nro, dtype: int64

In [ ]:
for idx in range(1,4):
  print('Amostra',idx)
  dfx = pd.read_csv('./amostra_news_integrada/amostra_'+str(idx)+'/train.csv',header=0,doublequote=True, quotechar='"', encoding='utf-8')
  print('Quantidade Categorias',len(dfx['category_nro'].value_counts()))

Amostra 1
Quantidade Categorias 32
Amostra 2
Quantidade Categorias 18
Amostra 3
Quantidade Categorias 32


In [ ]:
dfx['category_nro'].value_counts()

0     5208
1     4976
2     4885
3     4605
4     3980
5     3884
6     3699
7     1227
8     1068
9      982
10     883
11     545
12     487
13     431
14     428
15     309
16     298
18     230
17     222
21     196
19     196
20     193
22     164
24     146
23     137
25     104
26      90
27      89
28      78
29      44
31      32
30      30
32      25
33      20
38      14
34      12
37      11
36      10
35       7
42       6
39       6
41       6
45       4
47       4
40       4
43       3
52       2
46       2
44       2
94       1
54       1
77       1
62       1
56       1
79       1
61       1
53       1
70       1
57       1
92       1
48       1
69       1
72       1
49       1
59       1
Name: category_nro, dtype: int64

In [ ]:
!ls ./amostra_news_integrada/amostra_2

test.csv  train.csv  valid.csv


In [ ]:
dfx = pd.read_csv("./amostra_news_integrada//amostra_2/train.csv",header=0,doublequote=True, quotechar='"', encoding='utf-8')
dfx.shape

(14400, 2)

In [ ]:
dfx.columns


Index(['text', 'category_nro'], dtype='object')

In [ ]:
dfx['tamanho']=dfx['text'].str.len()
dfx['tamanho'].describe()

count    14361.000000
mean      3052.520577
std       2547.915361
min          1.000000
25%       1483.000000
50%       2564.000000
75%       3844.000000
max      46545.000000
Name: tamanho, dtype: float64